<a href="https://colab.research.google.com/github/cristianbossolasco/scraper-agencias-viajes/blob/main/scraper_busplus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By



```
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

url = "http://example.com" 
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")

options.headless = True

driver = webdriver.Chrome("/usr/bin/chromedriver", options=options)

driver.get(url)
print(driver.title)
driver.quit()

```




In [3]:
origen_parada=28
destino_parada=176
fecha_ida='2023/01/28'

url = f'https://checkout.busplus.com.ar/servicios?origen_parada={origen_parada}&destino_parada={destino_parada}&fecha_ida={fecha_ida}&fecha_vuelta=&pasajeros=1&empresa=&cupondescuento='

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")

options.headless = True

driver = webdriver.Chrome("/usr/bin/chromedriver", options=options)

driver.get(url)
#print(driver.title)
#driver.quit()

<ipython-input-3-5f5d490bc74f>:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/bin/chromedriver", options=options)


In [29]:
def get_data(origen_parada, destino_parada, fecha_ida):
  url = f'https://checkout.busplus.com.ar/servicios?origen_parada={origen_parada}&destino_parada={destino_parada}&fecha_ida={fecha_ida}&fecha_vuelta=&pasajeros=1&empresa=&cupondescuento='

  options = Options()
  options.add_argument("--headless")
  options.add_argument("--no-sandbox")

  options.headless = True

  driver = webdriver.Chrome("/usr/bin/chromedriver", options=options)

  driver.get(url)

  INDEX_EMPRESA = 0
  INDEX_SALIDA = 1
  INDEX_LLEGADA = 3
  INDEX_ESCALA = 4
  INDEX_DURACION = 5
  INDEX_CATEGORIA = 6
  INDEX_PRECIO = 8
  # get_attribute('innerHTML') para obtener el html

  items = []

  filas  = driver.find_elements(By.CLASS_NAME,"fila_servicio.servicio_fila.idatr")
  for fila in filas:
    if not "hide " in fila.get_attribute(name="class"):
      id = fila.get_attribute('id')
      list_td = fila.find_elements(By.TAG_NAME,"td")

      empresa_img = list_td[INDEX_EMPRESA].find_element(By.TAG_NAME, 'img').get_attribute('src')
      salida = list_td[INDEX_SALIDA].text.replace('\n', ' ')
      llegada = list_td[INDEX_LLEGADA].text.replace('\n', ' ')
      escala = list_td[INDEX_ESCALA].find_elements(By.TAG_NAME,"a")[0].get_attribute('text')
      duracion = list_td[INDEX_DURACION].get_attribute('innerHTML').strip()
      xpath_categoria = f'//*[@id="{id}"]/td[7]/ul/li[1]'
      categoria = driver.find_element(By.XPATH, xpath_categoria).text
      xpath_butaca = f'//*[@id="{id}"]/td[8]/ul/li[1]'
      butacas_libres = driver.find_element(By.XPATH, xpath_butaca).text
      xpath_precio = f'//*[@id="{id}"]/td[9]/ul/li[1]/span'
      precio = driver.find_element(By.XPATH, xpath_precio).text


      item_json = {
          "origen_parada": origen_parada,
          "destino_parada": destino_parada,
          "fecha_ida": fecha_ida,
          "empresa_img": empresa_img,
          "salida": salida,
          "llegada": llegada,
          "escala": escala,
          "duracion": duracion,
          "categoria": categoria,
          "butacas_libres": butacas_libres,
          "precio": precio
      }

      items.append(item_json)

  driver.quit()
  return items


In [30]:
get_data(28, 176, '2023/01/30')

<ipython-input-29-aa25ba0ef6bf>:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/bin/chromedriver", options=options)


[{'origen_parada': 28,
  'destino_parada': 176,
  'fecha_ida': '2023/01/30',
  'empresa_img': 'https://checkout.busplus.com.ar/images/v2/empresas/50.png',
  'salida': '11:15 Lun 30/01',
  'llegada': '18:10 Lun 30/01',
  'escala': 'Directo ',
  'duracion': '06:55hs',
  'categoria': 'Estandar',
  'butacas_libres': '37',
  'precio': '$5400'},
 {'origen_parada': 28,
  'destino_parada': 176,
  'fecha_ida': '2023/01/30',
  'empresa_img': 'https://checkout.busplus.com.ar/images/v2/empresas/50.png',
  'salida': '16:40 Lun 30/01',
  'llegada': '23:35 Lun 30/01',
  'escala': 'Directo ',
  'duracion': '06:55hs',
  'categoria': 'Semi Cama',
  'butacas_libres': '10',
  'precio': '$6250'},
 {'origen_parada': 28,
  'destino_parada': 176,
  'fecha_ida': '2023/01/30',
  'empresa_img': 'https://checkout.busplus.com.ar/images/v2/empresas/78.png',
  'salida': '12:30 Lun 30/01',
  'llegada': '22:20 Lun 30/01',
  'escala': 'Transbordo ',
  'duracion': '09:50hs',
  'categoria': 'Cama Ejecutivo',
  'butacas_l

In [6]:
list_td[INDEX_CATEGORIA].find_element(By.CLASS_NAME,"barato").get_attribute('text')
